## Backtesting crypto arbitrage strategy

### Summary
In this notebook, we will try to find the best crypto pair and the best exchange pair 

### Go to project root

In [1]:
import os
from pathlib import Path

#list the current work dir
cwd = os.getcwd()
current_path = Path(cwd)
project_root = current_path.parent.parent

#change the current work dir
os.chdir(project_root)

### Imports

In [2]:
from collections import deque
from datetime import datetime, timedelta, timezone

from trazy_analysis.bot.event_loop import EventLoop
from trazy_analysis.broker.binance_fee_model import BinanceFeeModel
from trazy_analysis.broker.broker_manager import BrokerManager
from trazy_analysis.broker.kucoin_fee_model import KucoinFeeModel
from trazy_analysis.broker.percent_fee_model import PercentFeeModel
from trazy_analysis.broker.simulated_broker import SimulatedBroker
from trazy_analysis.common.clock import SimulatedClock
from trazy_analysis.common.ccxt_connector import CcxtConnector
from trazy_analysis.feed.feed import CsvFeed, Feed, ExternalStorageFeed
from trazy_analysis.indicators.indicators_manager import IndicatorsManager
from trazy_analysis.models.asset import Asset
from trazy_analysis.models.candle import Candle
from trazy_analysis.models.enums import Action, Direction, OrderType
from trazy_analysis.models.order import Order
from trazy_analysis.order_manager.order_creator import OrderCreator
from trazy_analysis.order_manager.order_manager import OrderManager
from trazy_analysis.order_manager.position_sizer import PositionSizer
from trazy_analysis.strategy.strategies.arbitrage_strategy import ArbitrageStrategy
from trazy_analysis.market_data.historical.binance_historical_data_handler import BinanceHistoricalDataHandler
from trazy_analysis.market_data.historical.kucoin_historical_data_handler import KucoinHistoricalDataHandler
from trazy_analysis.market_data.historical.ccxt_historical_data_handler import CcxtHistoricalDataHandler
from trazy_analysis.common.helper import get_or_create_nested_dict
from trazy_analysis.db_storage.mongodb_storage import MongoDbStorage
from trazy_analysis.db_storage.influxdb_storage import InfluxDbStorage

import numpy as np
import pandas as pd
import ccxt
import re
import multiprocessing
import pytz
import tqdm

### Define constants

In [ ]:
mongo_db_storage = MongoDbStorage()

In [ ]:
result = mongo_db_storage.get_state("ccxt_avalaible_exchanges")
result

In [ ]:
# get list of exchanges and create api_keys
blacklisted = set()#{"bitbay"}
exchanges = ccxt.exchanges
exchanges = [exchange for exchange in exchanges if exchange not in blacklisted]
mongo_
exchanges_api_keys = {
    exchange: {
        "key": None,
        "secret": None,
        "password": None,
    } for exchange in exchanges
}

ccxt_connector = CcxtConnector(exchanges_api_keys=exchanges_api_keys)

In [ ]:
# Build FeeModels

fee_models = {}
to_process = len(exchanges)
processed = 0
uniform_symbol_to_original_symbol = {}

# Tickers format
format1 = re.compile('^[a-zA-Z0-9_]+/[a-zA-Z0-9_]+$')
format2 = re.compile('^\.[a-zA-Z0-9_]+$')
format3 = re.compile('^\$[a-zA-Z0-9_]+/[a-zA-Z0-9_]+$')
format4 = re.compile('^[a-zA-Z0-9_]+-[a-zA-Z0-9_]+$')
format5 = re.compile('^[a-zA-Z0-9_]+FP$')
format6 = re.compile('^[a-zA-Z0-9_]+_BQX$')
format7 = re.compile('^[a-zA-Z0-9_]+_[0-9]{6}$')
format8 = re.compile('^CMT_[a-zA-Z0-9_]+$')
format9 = re.compile('^[a-zA-Z0-9_]+-[a-zA-Z0-9_]+-[0-9]{6}$')
format10 = re.compile('^[a-zA-Z0-9_]+-[a-zA-Z0-9_]+-SWAP$')


for exchange in exchanges:
    print(f"Building fee models for exchange {exchange}")
    exchange_to_lower = exchange.lower()
    exchange_instance = ccxt_connector.get_exchange_instance(exchange_to_lower)

    # first check if we can retrieve historical data
    if "fetchOHLCV" not in exchange_instance.has or not exchange_instance.has["fetchOHLCV"]:
        processed += 1
        print(f"Progress: {processed} / {to_process}")
        continue
    if "fetchMarkets" not in exchange_instance.has or not exchange_instance.has["fetchMarkets"]:
        processed += 1
        print(f"Progress: {processed} / {to_process}")
        continue
    try:
        market_info = exchange_instance.fetchMarkets()
    except Exception as e:
        processed += 1
        print(f"Progress: {processed} / {to_process}")
        continue
    get_or_create_nested_dict(fee_models, exchange)


    for symbol_info in market_info:
        symbol = symbol_info["symbol"]
        symbol_before = symbol
        if format1.match(symbol) is not None:
            symbol = symbol.replace("/", "").upper()
        elif format2.match(symbol) is not None:
            symbol = symbol.replace(".", "").upper()
        elif format3.match(symbol) is not None:
            symbol = symbol.replace("$", "").upper()
        elif format4.match(symbol) is not None:
            symbol = symbol.replace("-", "").upper()
        elif format5.match(symbol) is not None:
            symbol = symbol[:-2].upper()
        elif format6.match(symbol) is not None:
            symbol = symbol[:-4].upper()
        elif format7.match(symbol) is not None:
            symbol = symbol[:-7].upper()
        elif format8.match(symbol) is not None:
            symbol = symbol[:4].upper()
        elif format9.match(symbol) is not None:
            symbol = symbol.replace("-", "")[:-6].upper()
        elif format10.match(symbol) is not None:
            symbol = symbol.replace("-", "")[:-4].upper()
        else:
            continue
        

        get_or_create_nested_dict(fee_models, exchange, symbol)
        # to simplify we just take the maximum of the 2 fees
        # print(symbol_info)
        if "maker" not in symbol_info and "taker" not in symbol_info:
            continue
        maker_fee = taker_fee = 0
        if "maker" in symbol_info and symbol_info["maker"] is not None:
            maker_fee = float(symbol_info["maker"])
        if "taker" in symbol_info and symbol_info["taker"] is not None:
            taker_fee = float(symbol_info["taker"])
        fee = max(maker_fee, taker_fee)
        fee_models[exchange][symbol] = PercentFeeModel(commission_pct=fee)
        get_or_create_nested_dict(uniform_symbol_to_original_symbol, exchange)
        uniform_symbol_to_original_symbol[exchange][symbol] = symbol_before
    processed += 1
    print(f"Progress: {processed} / {to_process}")

In [ ]:
# build exchange pairs
seen = set()
exchange_pairs = []
filtered_exchanges = list(uniform_symbol_to_original_symbol.keys())
for exchange1 in filtered_exchanges:
    for exchange2 in filtered_exchanges:
        if exchange2 in seen or exchange1 == exchange2:
            continue
        exchange_pairs.append((exchange1, exchange2))
    seen.add(exchange1)

LOOKBACK_PERIOD = timedelta(days=1)

MINIMUM_TRANSACTIONS = 5
MINIMUM_VOLUME_IN_CASH = 100
STABLE_COINS = ["USDT", "USDC", "BUSD", "DAI", "UST", "TUSD", "PAX", "HUSD", "USDN", "GUSD"]

### Find common crypto pairs between exchange pairs

In [ ]:
# Find common pairs

all_exchange_pairs = {}
for exchange in filtered_exchanges:
    all_exchange_pairs[exchange] = set()
    
common_pairs_dict = {}
for exchange_pair in exchange_pairs:
    exchange1 = exchange_pair[0]
    exchange2 = exchange_pair[1]
    get_or_create_nested_dict(common_pairs_dict, exchange1, exchange2)
    
    exchange1_tickers_list = list(uniform_symbol_to_original_symbol[exchange1].keys())
    exchange2_tickers_list = list(uniform_symbol_to_original_symbol[exchange2].keys())
    
    common_pairs = np.intersect1d(exchange1_tickers_list, exchange2_tickers_list)
    
    def ends_with_stable_coin(pair: str):
        for stable_coin in STABLE_COINS:
            if pair.endswith(stable_coin):
                return True
        return False

    # Consider only stable coin pairs for now
    common_pairs = [common_pair for common_pair in common_pairs if ends_with_stable_coin(common_pair)]

    all_exchange_pairs[exchange1] |= set(common_pairs)
    all_exchange_pairs[exchange2] |= set(common_pairs)

    common_pairs_dict[exchange1][exchange2] = common_pairs

In [ ]:
all_exchange_pairs_filtered = {
    exchange: pairs for exchange, pairs in all_exchange_pairs.items()
    if len(pairs) != 0
}
filtered_exchanges = list(all_exchange_pairs_filtered.keys())

In [ ]:
all_exchange_pairs_filtered

In [ ]:
common_pairs_dict

### Download Data for the last lookback period

In [4]:
LOOKBACK_PERIOD = timedelta(days=1)
end = datetime.now(pytz.UTC)
start = end - LOOKBACK_PERIOD

In [5]:
start

datetime.datetime(2021, 8, 7, 19, 7, 4, 190316, tzinfo=<UTC>)

In [7]:
end

datetime.datetime(2021, 8, 8, 19, 7, 4, 190316, tzinfo=<UTC>)

In [ ]:
start = datetime(2021, 7, 30, 21, 3, 48, 25463, tzinfo=pytz.UTC)
end = datetime(2021, 7, 31, 7, 3, 48, 25463, tzinfo=pytz.UTC)

In [ ]:
download_combinations = []
for exchange in filtered_exchanges:
    all_pairs = all_exchange_pairs_filtered[exchange]
    for pair in all_pairs:
        download_combinations.append((exchange, pair))
len_download_combinations = len(download_combinations)

In [ ]:
db_storage = InfluxDbStorage()
historical_data_handler = CcxtHistoricalDataHandler(ccxt_connector)

def download_data(download_combination):
    exchange, pair = download_combination[0], download_combination[1]
    print(f"Downloading {exchange}-{pair}")
    original_pair = uniform_symbol_to_original_symbol[exchange][pair]
    exchange_asset = Asset(symbol=original_pair, exchange=exchange)
    historical_data_handler.save_ticker_data_in_db_storage(
        exchange_asset, db_storage, start, end
    )
    print(f"Finished downloading {exchange}-{pair}")

In [ ]:
pool = multiprocessing.Pool()
for _ in tqdm.tqdm(pool.imap_unordered(download_data, download_combinations), total=len_download_combinations):
    pass

In [ ]:
combinations = []
for exchange_pair in exchange_pairs:
    exchange1 = exchange_pair[0]
    exchange2 = exchange_pair[1]

    common_pairs = common_pairs_dict[exchange1][exchange2]
    for common_pair in common_pairs:
        combinations.append((exchange1, exchange2, common_pair))
initial_budget = 2000
empty_result = {
    "profit_pct": 0,
    "cash_profit_pct": 0,
    "coefficient of varition (abs)": 1000000,
    "nb_errors": 0,
    "errors_pct": 0,
    "minimum number of transactions in both exchanges": 0,
    "avg_volume_in_cash": 0,
    "median_volume_in_cash": 0
}
len_combinations = len(combinations)
#combinations=[('aax', 'probit', 'REPUSDT'), ('aax', 'probit', 'REPUSDT')]

In [ ]:
combinations

In [ ]:
def process_common_pair_exchanges_pair(combination):
    exchange1, exchange2, common_pair = combination[0], combination[1], combination[2]
    exchanges = [exchange1, exchange2]
    common_pair_key = f"{exchange1}_{exchange2}_{common_pair}"
    print(f"Checking arbitrage opportunities for {common_pair}")
    events = deque()
    common_pair1 = uniform_symbol_to_original_symbol[exchange1][common_pair]
    common_pair2 = uniform_symbol_to_original_symbol[exchange2][common_pair]
    assets = [
        Asset(symbol=common_pair1, exchange=exchange1),
        Asset(symbol=common_pair2, exchange=exchange2)
    ]
    assets_dict = {asset.exchange: asset for asset in assets}

    # load data
    #feed: Feed = CsvFeed(
    #    {
    #        asset: f"backtests/crypto_arbitrage/data/{common_pair.lower()}_{asset.exchange.lower()}.csv"
    #        for asset in assets
    #    },
    #    events,
    #)

    feed: Feed = ExternalStorageFeed(
        assets = assets,
        events = events,
        time_unit = timedelta(minutes=1),
        start = start,
        end = end,
        db_storage = db_storage,
        file_storage = None,
        market_cal = None,
    )

    # Check wether data is empty or not
    exchange1_candle_dataframe = feed.candle_dataframes[
        assets_dict[exchange1]
    ]
    if exchange1_candle_dataframe.empty:
        print(f"{common_pair_key} data is empty for exchange1_candle_dataframe so it is skipped.")
        return empty_result

    exchange2_candle_dataframe = feed.candle_dataframes[
        assets_dict[exchange2]
    ]
    #print(exchange2_candle_dataframe)
    if exchange2_candle_dataframe.empty:
        print(f"{common_pair_key} data is empty for exchange2_candle_dataframe so it is skipped.")
        return empty_result

    # don't process the pair if the volume is low
    exchange1_prices = pd.to_numeric(exchange1_candle_dataframe["close"])
    exchange1_volumes = pd.to_numeric(exchange1_candle_dataframe["volume"])
    exchange1_avg_price = exchange1_prices.mean()
    exchange1_median_price = exchange1_prices.median()
    exchange1_avg_volume = exchange1_volumes.mean()
    exchange1_median_volume = exchange1_volumes.median()

    exchange2_prices = pd.to_numeric(exchange2_candle_dataframe["close"])
    exchange2_volumes = pd.to_numeric(exchange2_candle_dataframe["volume"])
    exchange2_avg_price = exchange2_prices.mean()
    exchange2_median_price = exchange2_prices.median()
    exchange2_avg_volume = exchange2_volumes.mean()
    exchange2_median_volume = exchange2_volumes.median()

    avg_price = min(exchange1_avg_price, exchange2_avg_price)
    median_price = min(exchange1_median_price, exchange2_median_price)
    avg_volume = min(exchange1_avg_volume, exchange2_avg_volume)
    median_volume = min(exchange1_median_volume, exchange2_median_volume)

    avg_volume_in_cash = avg_volume * avg_price
    median_volume_in_cash = median_volume * median_price

    # if volume_in_cash == 0.0:
    #    processed += 1
    #    print(f"{common_pair_key} volume in cash {volume_in_cash} is lower than the minimum volume in cash required {MINIMUM_VOLUME_IN_CASH} so it is skipped.")
    #    print(f"Current rank: {profit_rank}")
    #    print(f"Progress: {processed} / {to_process}")
    #    continue

    # Create brokers for exchanges, put a big amount of cash and a big amount of shares to allow all two
    # ways transactions
    strategies = {ArbitrageStrategy: [{"margin_factor": 2}]}

    clock = SimulatedClock()
    initial_funds = initial_budget / 2
    # print(f"initial_funds = {initial_funds}")
    # Create brokers with a big amount of money
    exchange1_broker = SimulatedBroker(
        clock,
        events,
        initial_funds=initial_funds,
        fee_model=fee_models[exchange1][common_pair],
        exchange=exchange1,
    )
    exchange1_broker.subscribe_funds_to_portfolio(initial_funds)
    exchange1_first_candle = exchange1_candle_dataframe.get_candle(0)
    exchange1_broker.update_price(exchange1_first_candle)

    exchange2_broker = SimulatedBroker(
        clock,
        events,
        initial_funds=initial_funds,
        fee_model=fee_models[exchange2][common_pair],
        exchange=exchange2,
    )
    # exchange2_broker.subscribe_funds_to_portfolio(initial_funds)
    exchange2_first_candle = exchange2_candle_dataframe.get_candle(0)
    exchange2_broker.update_price(exchange2_first_candle)
    max_size_exchange1 = exchange1_broker.max_entry_order_size(
        assets_dict[exchange1], Direction.LONG, initial_funds
    )
    max_size_exchange2 = exchange2_broker.max_entry_order_size(
        assets_dict[exchange2], Direction.LONG, initial_funds
    )
    initial_size = max_size_exchange2

    # exchange 2
    candle = Candle(
        asset=assets_dict[exchange2], open=0, high=0, low=0, close=0, volume=0
    )
    exchange2_broker.update_price(candle)
    order = Order(
        asset=assets_dict[exchange2],
        action=Action.BUY,
        direction=Direction.LONG,
        size=initial_size,
        signal_id="0",
        limit=None,
        stop=None,
        target=None,
        stop_pct=None,
        type=OrderType.MARKET,
        clock=clock,
        time_in_force=timedelta(minutes=5),
    )
    exchange2_broker.execute_market_order(order)

    # prepare event loop parameters
    broker_manager = BrokerManager(
        brokers={
            exchange1: exchange1_broker,
            exchange2: exchange2_broker,
        },
        clock=clock,
    )
    position_sizer = PositionSizer(broker_manager=broker_manager, integer_size=False)
    order_creator = OrderCreator(broker_manager=broker_manager)
    order_manager = OrderManager(
        events=events,
        broker_manager=broker_manager,
        position_sizer=position_sizer,
        order_creator=order_creator,
    )
    indicators_manager = IndicatorsManager(
        preload=True, initial_data=feed.candles
    )
    event_loop = EventLoop(
        events=events,
        assets=assets,
        feed=feed,
        order_manager=order_manager,
        strategies_parameters=strategies,
        indicators_manager=indicators_manager,
        close_at_end_of_day=False,
        close_at_end_of_data=False
    )

    # get initial state of portfolio for stats computation total_market_value
    exchange1_broker.update_price(exchange1_first_candle)
    exchange2_broker.update_price(exchange2_first_candle)
    initial_market_values = sum([broker_manager.get_broker(exchange).get_portfolio_total_market_value() for exchange in exchanges])
    initial_cash_balances = sum([broker_manager.get_broker(exchange).get_portfolio_cash_balance() for exchange in exchanges])
    initial_equities = initial_cash_balances + initial_market_values

    # print(f"initial_market_values = {initial_market_values}")
    # print(f"initial_cash_balances = {initial_cash_balances}")
    # print(f"initial_equities = {initial_equities}")

    event_loop.loop()
    
    exchange1_history = exchange1_broker.portfolio.history
    # print([str(portfolio_event) for portfolio_event in exchange1_history])
    exchange1_transactions = [portfolio_event for portfolio_event in exchange1_history if portfolio_event.type == "symbol_transaction"]
    # remove first transaction, which is to add securities in the broker
    exchange1_transactions = exchange1_transactions[1:]

    exchange2_history = exchange2_broker.portfolio.history
    # print([str(portfolio_event) for portfolio_event in exchange2_history])
    exchange2_transactions = [portfolio_event for portfolio_event in exchange2_history if portfolio_event.type == "symbol_transaction"]
    # remove first transaction, which is to add securities in the broker
    exchange2_transactions = exchange2_transactions[1:]

    # exchange1_transactions_timestamps = np.array([transaction.timestamp for transaction in exchange1_transactions], dtype=datetime)
    # exchange2_transactions_timestamps = np.array([transaction.timestamp for transaction in exchange2_transactions], dtype=datetime)
    # common_timestamps = np.intersect1d(exchange1_transactions_timestamps, exchange2_transactions_timestamps)

    # filter timestamps
    # exchange1_transactions = [transaction for transaction in exchange1_transactions if transaction.timestamp in common_timestamps]
    # exchange2_transactions = [transaction for transaction in exchange2_transactions if transaction.timestamp in common_timestamps]

    nb_transactions1 = len(exchange1_transactions)
    nb_transactions2 = len(exchange2_transactions)
    min_nb_transactions = min(nb_transactions1, nb_transactions2)      
    #if min_nb_transactions < MINIMUM_TRANSACTIONS:
    #    processed += 1
    #    print(f"{common_pair_key} has only {min_nb_transactions} transactions which is less than the minimum required number of transactions {MINIMUM_TRANSACTIONS} so it is skipped.")
    #    print(f"Current rank: {profit_rank}")
    #    print(f"Progress: {processed} / {to_process}")
    #    continue

    # find missed opportunities

    """
    When an arbitrage opportunity is found, we submit 2 orders to the brokers of the 2 exchanges.
    If one of the order is not executed because of for example not enough cash or, whathever reason,
    we call it an "error". The less errors you have, the better it is for ensuring the stability of the strategy
    """
    nb_errors = 0
    transaction_profits = []
    i = 0
    j = 0

    while i < nb_transactions1 and j < nb_transactions2:
        transaction1 = exchange1_transactions[i]
        transaction2 = exchange2_transactions[j]
        if transaction1.timestamp == transaction2.timestamp:
            i += 1
            j += 1
        elif transaction1.timestamp < transaction2.timestamp:
            i += 1
            nb_errors += 1
        else: # transaction1.timestamp > transaction2.timestamp
            j += 1
            nb_errors += 1
        if transaction1.action == Action.BUY:
            transaction_profit = transaction2.credit - transaction1.debit
        else:
            transaction_profit = transaction1.credit - transaction2.debit
        transaction_profits.append(transaction_profit)

    #if nb_errors > min_nb_transactions:
    #    print(f"{common_pair_key} nb errors is greater than number of transactions {min_nb_transactions} transactions so it is skipped.")
    #    continue

    volume_in_cash_result = {
        "profit_pct": 0,
        "cash_profit_pct": 0,
        "coefficient of varition (abs)": 1000000,
        "nb_errors": 0,
        "errors_pct": 0,
        "minimum number of transactions in both exchanges": min_nb_transactions,
        "avg_volume_in_cash": avg_volume_in_cash,
        "median_volume_in_cash": median_volume_in_cash
    }
    if len(transaction_profits) == 0:
        print(f"{common_pair_key} transaction_profits is zero so it is skipped.")
        return volume_in_cash_result

    errors_pct = 0
    max_nb_transactions = max(nb_transactions1, nb_transactions2)
    if max_nb_transactions != 0:
        errors_pct = nb_errors / max_nb_transactions * 100
    cash_profit = sum(transaction_profits)

    # let's find the coefficient of variation to filter
    cv = lambda x: np.std(x, ddof=1) / np.mean(x) * 100 
    coefficient_of_variation = cv(transaction_profits)

    final_market_values = sum([broker_manager.get_broker(exchange).get_portfolio_total_market_value() for exchange in exchanges])
    final_cash_balances = sum([broker_manager.get_broker(exchange).get_portfolio_cash_balance() for exchange in exchanges])
    final_equities = final_market_values + final_cash_balances

    # print(f"final_market_values = {final_market_values}")
    # print(f"final_cash_balances = {final_cash_balances}")
    # print(f"final_equities = {final_equities}")

    profit = final_equities - initial_equities

    profit_pct = profit / initial_equities * 100

    cash_profit_pct = cash_profit / initial_cash_balances * 100

    return {
        "profit_pct": profit_pct,
        "cash_profit_pct": cash_profit_pct,
        "coefficient of varition (abs)": abs(coefficient_of_variation),
        "nb_errors": nb_errors,
        "errors_pct": errors_pct,
        "minimum number of transactions in both exchanges": min_nb_transactions,
        "avg_volume_in_cash": avg_volume_in_cash,
        "median_volume_in_cash": median_volume_in_cash
    }

    # exchange1_broker.portfolio.history_to_df().to_csv(f"backtests/crypto_arbitrage/data/transactions/{common_pair.lower()}_{exchange1.lower()}_transactions.csv")
    # exchange2_broker.portfolio.history_to_df().to_csv(f"backtests/crypto_arbitrage/data/transactions/{common_pair.lower()}_{exchange2.lower()}_transactions.csv")

    # Order by number of errors
    # profit_rank = {k: v for k, v in sorted(profit_pct_dict.items(), key=lambda item: item[1]["coefficient of varition (abs)"], reverse=False)}
    # print(f"Current rank: {profit_rank}")
    # best_results_df = pd.DataFrame.from_dict(profit_rank, orient="index")
    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #    print(best_results_df)

In [ ]:
results_dict = {}

pool = multiprocessing.Pool()
results = list(
    tqdm.tqdm(
        pool.imap_unordered(
            process_common_pair_exchanges_pair,
            combinations
        ),
        total=len(combinations)
    )
)

print(results)

In [ ]:
results_dict = {
    f"{combination[0]}-{combination[1]}-{combination[2]}": results[index]
    for index, combination in enumerate(combinations)
}
best_results_df = pd.DataFrame.from_dict(results_dict, orient="index")

In [ ]:
best_results_df.to_csv("final_results.csv")

In [8]:
dtype = {
    "profit_pct": float,
    "cash_profit_pct": float,
    "coefficient of varition (abs)": float,
    "nb_errors": int,
    "errors_pct": float,
    "minimum number of transactions in both exchanges": int,
    "avg_volume_in_cash": float,
    "median_volume_in_cash": float
}
best_results_df = pd.read_csv("final_results.csv", dtype=dtype)

In [10]:
best_results_df.to_dict()

{'Unnamed: 0': {0: 'aax-aofex-AAVEUSDT',
  1: 'aax-aofex-ADAUSDT',
  2: 'aax-aofex-AXSUSDT',
  3: 'aax-aofex-BATUSDT',
  4: 'aax-aofex-BCHUSDT',
  5: 'aax-aofex-BSVUSDT',
  6: 'aax-aofex-BTCUSDT',
  7: 'aax-aofex-BTTUSDT',
  8: 'aax-aofex-CAKEUSDT',
  9: 'aax-aofex-CHZUSDT',
  10: 'aax-aofex-CTXCUSDT',
  11: 'aax-aofex-DASHUSDT',
  12: 'aax-aofex-DOCKUSDT',
  13: 'aax-aofex-DOGEUSDT',
  14: 'aax-aofex-DOTUSDT',
  15: 'aax-aofex-ELFUSDT',
  16: 'aax-aofex-ENJUSDT',
  17: 'aax-aofex-EOSUSDT',
  18: 'aax-aofex-ETCUSDT',
  19: 'aax-aofex-ETHUSDT',
  20: 'aax-aofex-FILUSDT',
  21: 'aax-aofex-LINKUSDT',
  22: 'aax-aofex-LPTUSDT',
  23: 'aax-aofex-LRCUSDT',
  24: 'aax-aofex-LTCUSDT',
  25: 'aax-aofex-MANAUSDT',
  26: 'aax-aofex-MATICUSDT',
  27: 'aax-aofex-NEOUSDT',
  28: 'aax-aofex-OMGUSDT',
  29: 'aax-aofex-QTUMUSDT',
  30: 'aax-aofex-RENUSDT',
  31: 'aax-aofex-REPUSDT',
  32: 'aax-aofex-SANDUSDT',
  33: 'aax-aofex-SHIBUSDT',
  34: 'aax-aofex-SOLUSDT',
  35: 'aax-aofex-THETAUSDT',
  36: 'aa

In [ ]:
print(best_results_df.columns)
print(len(best_results_df))

In [ ]:
mask = (best_results_df["profit_pct"] > 0) & \
       (best_results_df["cash_profit_pct"] > 0)
best_results_df = best_results_df[mask].dropna()
len(best_results_df)

In [ ]:
best_results_df = best_results_df.sort_values("profit_pct")

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [ ]:
weights =  {
    "profit_pct": 5,
    "cash_profit_pct": 3,
    "coefficient of varition (abs)": 2,
    "nb_errors": 0,
    "errors_pct": 2,
    "minimum number of transactions in both exchanges": 3,
    "avg_volume_in_cash": 0,
    "median_volume_in_cash": 0
}
import math

def get_score(row):
    score = 0
    for metric, weight in weights.items():
        score += math.exp(weight) * row[metric]
    return score

In [ ]:
best_results_df["score"] = best_results_df.apply(get_score, axis=1)

In [ ]:
best_results_df.sort_values("profit_pct")